In [1]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install easyocr
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.3/977.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import cv2
import numpy as np
import sys
from ultralytics import YOLO
from sklearn.cluster import KMeans

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def get_grass_color(img):
    """
    Finds the color of the grass in the background of the image

    """
    # Convert image to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define range of green color in HSV
    lower_green = np.array([30, 40, 40])
    upper_green = np.array([80, 255, 255])

    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Calculate the mean value of the pixels that are not masked
    masked_img = cv2.bitwise_and(img, img, mask=mask)
    grass_color = cv2.mean(img, mask=mask)
    return grass_color[:3]

In [4]:
def get_kits_colors(players, grass_hsv=None, frame=None):
  """
  Finds the kit colors of all the players in the current frame

  """
  kits_colors = []
  if grass_hsv is None:
	    grass_color = get_grass_color(frame)
	    grass_hsv = cv2.cvtColor(np.uint8([[list(grass_color)]]), cv2.COLOR_BGR2HSV)

  for player_img in players:
      # Convert image to HSV color space
      hsv = cv2.cvtColor(player_img, cv2.COLOR_BGR2HSV)

      # Define range of green color in HSV
      lower_green = np.array([grass_hsv[0, 0, 0] - 10, 40, 40])
      upper_green = np.array([grass_hsv[0, 0, 0] + 10, 255, 255])

      # Threshold the HSV image to get only green colors
      mask = cv2.inRange(hsv, lower_green, upper_green)

      # Bitwise-AND mask and original image
      mask = cv2.bitwise_not(mask)
      upper_mask = np.zeros(player_img.shape[:2], np.uint8)
      upper_mask[0:player_img.shape[0]//2, 0:player_img.shape[1]] = 255
      mask = cv2.bitwise_and(mask, upper_mask)

      kit_color = np.array(cv2.mean(player_img, mask=mask)[:3])

      kits_colors.append(kit_color)
  return kits_colors

def get_kits_classifier(kits_colors):
  """
  Creates a K-Means classifier that can classify the kits accroding to their BGR
  values into 2 different clusters each of them represents one of the teams

  """
  kits_kmeans = KMeans(n_clusters=2)
  kits_kmeans.fit(kits_colors);
  return kits_kmeans

In [5]:
import cv2
import re
import easyocr
import numpy as np
import pandas as pd
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort

def annotate_video2(video_path, model):
    # DeepSORT 초기화
    deepsort = DeepSort(max_age=30, n_init=3, nn_budget=70)

    # 비디오 캡처 초기화
    cap = cv2.VideoCapture(video_path)

    # 원본 비디오 속성 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)  # 원본 FPS 가져오기
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 전체 프레임 수
    size = (width, height)

    # 결과 비디오 초기화
    output_video = cv2.VideoWriter('결과물입니당_ㅎㅇ.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

    # EasyOCR Reader 초기화
    reader = easyocr.Reader(['en'], gpu=True)

    # 등번호 히스토리 초기화
    track_number_history = {}

    # 프레임별 추적 정보 저장 리스트
    tracking_data = []

    # tqdm으로 진행 상황 표시
    with tqdm(total=total_frames, desc="Processing Video Frames") as pbar:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            current_frame_idx = cap.get(cv2.CAP_PROP_POS_FRAMES)
            frame_time = current_frame_idx / fps

            # YOLO 탐지
            result = model(frame, conf=0.5, verbose=False)[0]

            # YOLO 결과 변환 (DeepSORT 입력 형식으로 변환)
            detections = []
            yolo_boxes = []
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                confidence = float(box.conf.cpu().numpy()[0])
                detections.append([np.array([x1, y1, x2, y2]), confidence])
                yolo_boxes.append((x1, y1, x2, y2))

            # 등번호 탐지
            detected_numbers = {}
            for idx, (x1, y1, x2, y2) in enumerate(yolo_boxes):
                player_crop = frame[y1:y2, x1:x2]
                ocr_results = reader.readtext(player_crop)
                player_number = "N/A"
                for (bbox, text, conf) in ocr_results:
                    if conf > 0.5:
                        match = re.search(r'\d+', text)
                        if match:
                            player_number = match.group()
                            break
                detected_numbers[idx] = player_number

            # 라벨 분류
            detected_labels = {}
            players_imgs = []
            players_indices = []

            for idx, (x1, y1, x2, y2) in enumerate(yolo_boxes):
                class_id = result.boxes[idx].cls.cpu().numpy().astype(int)

                if class_id == 0:
                    detected_labels[idx] = {"label": "Player", "color": (0, 255, 0)}
                    player_img = frame[y1:y2, x1:x2]
                    if player_img.size > 0:
                        players_imgs.append(player_img)
                        players_indices.append(idx)
                elif class_id == 1:
                    detected_labels[idx] = {"label": "Goalkeeper", "color": (0, 165, 255)}
                elif class_id == 2:
                    detected_labels[idx] = {"label": "Ball", "color": (0, 255, 255)}
                elif class_id == 3:
                    detected_labels[idx] = {"label": "Main Ref", "color": (123, 174, 213)}
                elif class_id == 4:
                    detected_labels[idx] = {"label": "Side Ref", "color": (217, 89, 204)}
                elif class_id == 5:
                    detected_labels[idx] = {"label": "Staff", "color": (22, 11, 15)}
                else:
                    detected_labels[idx] = {"label": "Unknown", "color": (255, 255, 255)}

            # 팀 분류
            if players_imgs:
                if current_frame_idx == 1:
                    grass_color = get_grass_color(frame)
                    grass_hsv = cv2.cvtColor(np.uint8([[list(grass_color)]]), cv2.COLOR_BGR2HSV)
                    kits_colors = get_kits_colors(players_imgs, grass_hsv)
                    kits_clf = get_kits_classifier(kits_colors)
                    left_team_label = 0
                else:
                    kits_colors = get_kits_colors(players_imgs, grass_hsv)

                team_labels = kits_clf.predict(kits_colors)
                for player_idx, team_label in zip(players_indices, team_labels):
                    detected_labels[player_idx]["team"] = "L" if team_label == left_team_label else "R"

            # DeepSORT 추적
            tracks = deepsort.update_tracks(detections, frame=frame)

            # DeepSORT와 YOLO 매핑
            yolo_id_mapping = {}
            for track in tracks:
                if not track.is_confirmed():
                    continue

                track_id = track.track_id
                ltrb = track.to_ltrb()
                x1_prime, y1_prime, x2_prime, y2_prime = map(int, ltrb)

                matched_idx = None
                min_distance = float('inf')
                for idx, (bx1, by1, bx2, by2) in enumerate(yolo_boxes):
                    distance = np.sqrt((x1_prime - bx1) ** 2 + (y1_prime - by1) ** 2)
                    if distance < min_distance:
                        min_distance = distance
                        matched_idx = idx

                if matched_idx is not None:
                    yolo_id_mapping[matched_idx] = track_id

            # 시각화 및 정보 저장
            for idx, (x1, y1, x2, y2) in enumerate(yolo_boxes):
                track_id = yolo_id_mapping.get(idx, None)
                player_number = detected_numbers.get(idx, "N/A")
                label_info = detected_labels.get(idx, {"label": "Unknown", "color": (255, 255, 255)})

                if track_id is not None:
                    if player_number != "N/A":
                        track_number_history[track_id] = player_number
                    else:
                        player_number = track_number_history.get(track_id, "N/A")

                # 바운딩 박스 시각화 색상 결정
                if label_info["label"] == "Player" and "team" in label_info:
                    box_color = (0, 255, 0) if label_info["team"] == "L" else (0, 0, 255)
                else:
                    box_color = label_info["color"]

                label_text = f"{label_info['label']}"
                if label_info["label"] == "Player" and "team" in label_info:
                    label_text += f" ({label_info['team']})"
                if track_id is not None:
                    label_text += f", ID: {track_id}"
                label_text += f", No: {player_number}"

                cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
                cv2.putText(frame, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)

                # 바운딩 박스 특성 계산
                width = x2 - x1
                height = y2 - y1
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2

                tracking_data.append({
                    "frame_idx": int(current_frame_idx),
                    "frame_time": round(frame_time, 3),
                    "track_id": track_id if track_id is not None else "N/A",
                    "number": player_number,
                    "team": label_info.get("team", "N/A"),
                    "role": label_info["label"],
                    "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                    "width": width, "height": height,
                    "x_center": x_center, "y_center": y_center
                })

            output_video.write(frame)
            pbar.update(1)

    cap.release()
    output_video.release()

    # 엑셀 저장
    df = pd.DataFrame(tracking_data)
    df.to_excel("frame_tracking_output.xlsx", index=False)
    print("엑셀 저장 완료: frame_tracking_output.xlsx")
    print("결과가 성공적으로 저장되었습니다! 행운을 빕니다,,,,")

In [6]:
model = YOLO("/content/last.pt")
video_path = '/content/유진.mov'
annotate_video2(video_path, model)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Processing Video Frames:  89%|████████▉ | 157/176 [00:54<00:06,  2.89it/s]


엑셀 저장 완료: frame_tracking_output.xlsx
결과가 성공적으로 저장되었습니다! 행운을 빕니다,,,,


In [7]:
cap = cv2.VideoCapture(video_path)

# 원본 비디오 속성 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(width, height)

2250 1258


In [8]:
df = pd.read_excel('frame_tracking_output.xlsx')

# 스케일 455, 256으로 맞추기

In [9]:
# 새 해상도
new_width, new_height = 455, 266

# 스케일 비율 계산
scale_x = new_width / width
scale_y = new_height / height


x_columns = ['x1', 'x2', 'x_center']
for col in x_columns:
    df[f'{col}_scaled'] = df[col] * scale_x

# y 관련 열
y_columns = ['y1', 'y2', 'y_center']
for col in y_columns:
    df[f'{col}_scaled'] = df[col] * scale_y

In [10]:
df

,frame_idx,frame_time,track_id,number,team,role,x1,y1,x2,y2,width,height,x_center,y_center,x1_scaled,x2_scaled,x_center_scaled,y1_scaled,y2_scaled,y_center_scaled
0,1,0.018,NaN,NaN,L,Player,1651,850,1679,925,28,75,1665.0,887.5,333.868889,339.531111,336.700000,179.729730,195.588235,187.658983
1,1,0.018,NaN,NaN,L,Player,1948,576,1993,639,45,63,1970.5,607.5,393.928889,403.028889,398.478889,121.793323,135.114467,128.453895
2,1,0.018,NaN,NaN,NaN,Staff,2014,1179,2053,1258,39,79,2033.5,1218.5,407.275556,415.162222,411.218889,249.295707,266.000000,257.647854
3,1,0.018,NaN,NaN,L,Player,2081,457,2118,520,37,63,2099.5,488.5,420.824444,428.306667,424.565556,96.631161,109.952305,103.291733
4,1,0.018,NaN,NaN,R,Player,1211,539,1248,594,37,55,1229.5,566.5,244.891111,252.373333,248.632222,113.969793,125.599364,119.784579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3733,157,2.869,23.0,NaN,NaN,Side Ref,2148,1172,2187,1253,39,81,2167.5,1212.5,434.373333,442.260000,438.316667,247.815580,264.942766,256.379173
3734,157,2.869,15.0,NaN,L,Player,683,523,712,585,29,62,697.5,554.0,138.117778,143.982222,141.050000,110.586645,123.696343,117.141494
3735,157,2.869,26.0,NaN,R,Player,647,269,669,326,22,57,658.0,297.5,130.837778,135.286667,133.062222,56.879173,68.931638,62.905405
3736,157,2.869,12.0,NaN,R,Player,1163,307,1188,359,25,52,1175.5,333.0,235.184444,240.240000,237.712222,64.914149,75.909380,70.411765


In [11]:
df.to_excel('22_video_frame_scaled.xlsx')